# BERT 전이학습

### 데이터 준비

In [ ]:
import pandas as pd
import os
from google.colab import files

# AIHub 감성대화 말뭉치 데이터셋 경로 안내
data_url = "https://aihub.or.kr/aihubdata/data/view.do?currMenu=&topMenu=&aihubDataSe=data&dataSetSn=86"

# 구글 코랩에서 파일 업로드
print("AIHub에서 감성대화 말뭉치 데이터를 다운로드한 후 업로드하세요:", data_url)

uploaded = files.upload()

# 업로드된 파일 확인
train_path = None
test_path = None
for filename in uploaded.keys():
    if 'train' in filename:
        train_path = filename
    elif 'test' in filename:
        test_path = filename

if not (train_path and test_path):
    raise FileNotFoundError("train 및 test 파일을 모두 업로드하세요.")


AIHub에서 감성대화 말뭉치 데이터를 다운로드한 후 업로드하세요: https://aihub.or.kr/aihubdata/data/view.do?currMenu=&topMenu=&aihubDataSe=data&dataSetSn=86


Saving test.xlsx to test.xlsx
Saving train.xlsx to train.xlsx


In [ ]:
# 데이터 로드
ratings_train_df = pd.read_excel(train_path)
ratings_test_df = pd.read_excel(test_path)

print(ratings_train_df.head())
print(ratings_test_df.head())

   Unnamed: 0  연령  성별     상황키워드  신체질환 감정_대분류 감정_소분류  \
0           1  청년  여성  진로,취업,직장  해당없음     분노  노여워하는   
1           2  청년  여성  진로,취업,직장  해당없음     분노  노여워하는   
2           3  청년  여성  진로,취업,직장  해당없음     분노  노여워하는   
3           4  청년  여성  진로,취업,직장  해당없음     분노  노여워하는   
4           5  청년  여성  진로,취업,직장  해당없음     분노  노여워하는   

                                               사람문장1  \
0                          일은 왜 해도 해도 끝이 없을까? 화가 난다.   
1     이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.   
2  회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...   
3  직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...   
4              얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.   

                                              시스템문장1  \
0                        많이 힘드시겠어요. 주위에 의논할 상대가 있나요?   
1           급여가 줄어 속상하시겠어요. 월급이 줄어든 것을 어떻게 보완하실 건가요?   
2  회사 동료 때문에 스트레스를 많이 받는 것 같아요. 문제 해결을 위해 어떤 노력을 ...   
3  관련 없는 심부름을 모두 하게 되어서 노여우시군요. 어떤 것이 상황을 나아질 수 있...   
4  무시하는 것 같은 태도에 화가 나셨군요. 상대방의 어떤 행동이 그런 감정을 유발하는...

In [ ]:
# 결측치 제거
ratings_train_df = ratings_train_df.dropna(how="any")
ratings_test_df = ratings_test_df.dropna(how="any")

display(ratings_train_df.info())
display(ratings_test_df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 42694 entries, 304 to 51629
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  42694 non-null  int64 
 1   연령          42694 non-null  object
 2   성별          42694 non-null  object
 3   상황키워드       42694 non-null  object
 4   신체질환        42694 non-null  object
 5   감정_대분류      42694 non-null  object
 6   감정_소분류      42694 non-null  object
 7   사람문장1       42694 non-null  object
 8   시스템문장1      42694 non-null  object
 9   사람문장2       42694 non-null  object
 10  시스템문장2      42694 non-null  object
 11  사람문장3       42694 non-null  object
 12  시스템문장3      42694 non-null  object
dtypes: int64(1), object(12)
memory usage: 4.6+ MB


None

<class 'pandas.core.frame.DataFrame'>
Index: 4686 entries, 0 to 6580
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  4686 non-null   int64 
 1   연령          4686 non-null   object
 2   성별          4686 non-null   object
 3   상황키워드       4686 non-null   object
 4   신체질환        4686 non-null   object
 5   감정_대분류      4686 non-null   object
 6   감정_소분류      4686 non-null   object
 7   사람문장1       4686 non-null   object
 8   시스템문장1      4686 non-null   object
 9   사람문장2       4686 non-null   object
 10  시스템문장2      4686 non-null   object
 11  사람문장3       4686 non-null   object
 12  시스템문장3      4686 non-null   object
dtypes: int64(1), object(12)
memory usage: 512.5+ KB


None

In [ ]:
# 감정_대분류 열의 고유 값 확인
print(ratings_train_df['감정_대분류'].unique())

# 감정_소분류 열의 고유 값 확인
print(ratings_train_df['감정_소분류'].unique())

['상처' '슬픔' '기쁨' '분노' '불안' '당황']
['상처' '충격 받은' '후회되는' '느긋' '자신하는' '좌절한' '노여워하는' '취약한' '외로운' '실망한' '불안'
 '걱정스러운' '신뢰하는' '남의 시선을 의식하는' '버려진' '눈물이 나는' '비통한' '안도' '죄책감의' '당황'
 '구역질 나는' '툴툴대는' '조심스러운' '성가신' '편안한' '혐오스러운' '짜증내는' '당혹스러운' '안달하는' '고립된'
 '열등감' '혼란스러운' '우울한' '회의적인' '흥분' '배신당한' '낙담한' '괴로워하는' '신이 난' '만족스러운'
 '감사하는' '환멸을 느끼는' '질투하는' '희생된' '한심한' '악의적인' '두려운' '부끄러운' '억울한' '분노' '초조한'
 '방어적인' '슬픔' '스트레스 받는' '염세적인' '가난한, 불우한' '기쁨' '마비된']


In [ ]:
# 감정 라벨 생성 (예: 부정 감정을 1, 긍정 감정을 0으로 변환)
negative_emotions = ['상처', '슬픔', '불안', '당황', '분노']
ratings_train_df['label'] = ratings_train_df['감정_대분류'].apply(lambda x: 1 if x in negative_emotions else 0)
ratings_test_df['label'] = ratings_test_df['감정_대분류'].apply(lambda x: 1 if x in negative_emotions else 0)

# 주요 텍스트 열 결합 (사람문장1 + 시스템문장1 + 사람문장2 + 시스템문장2 + 사람문장3 + 시스템문장3 사용)
text_columns = ['사람문장1', '시스템문장1', '사람문장2', '시스템문장2', '사람문장3', '시스템문장3']
ratings_train_df['document'] = ratings_train_df[text_columns].fillna('').agg(' '.join, axis=1)
ratings_test_df['document'] = ratings_test_df[text_columns].fillna('').agg(' '.join, axis=1)

# 데이터 샘플링 (학습 시간 단축용)
ratings_train_df = ratings_train_df.sample(n=8000, random_state=0)
ratings_test_df = ratings_test_df.sample(n=3000, random_state=0)

# 라벨 분포 확인
print(ratings_train_df['label'].value_counts(), ratings_test_df['label'].value_counts())

# print(ratings_train_df.head())

label
1    7083
0     917
Name: count, dtype: int64 label
1    2514
0     486
Name: count, dtype: int64


In [ ]:
X_train = ratings_train_df['document'].values.tolist()
y_train = ratings_train_df['label'].values.tolist()

X_test = ratings_test_df['document'].values.tolist()
y_test = ratings_test_df['label'].values.tolist()

In [ ]:
print(X_train[:5])
print(y_train[:5])

['다들 날 보면 수근 대는 거 같고 날 욕하는 느낌이야. 어떤 점에서 그런 생각을 하신 거예요? 딱히 수상한 거는 없는데 주변에서 다들 나를 욕하는 거 같아 아무도 못 믿겠어. 주변 분들이랑 얘기는 해보신 거예요? 아니 말 안 해도 느낌으로 알 수 있어 다들 왜 내 욕을 하는 거지? 주변 분과 대화를 시도해 보셨나요?', '엄마는 나를 걱정해서 충고를 했는데 나는 짜증만 냈어. 충고하는 엄마에게 짜증을 냈군요. 돌이켜보니 짜증을 낸 게 미안한데 사과하기도 애매한 것 같아. 사과하기가 애매하시군요. 어떻게 하는 것이 가장 좋을까요? 다시 생각해보니 진심으로 사과를 하는 것이 좋을 것 같아. 진심으로 사과하셔서 잘 해결되기를 바라요.', '아내가 빚을 남기고 죽었어. 정말 허탈해. 아내 때문에 허탈함을 느끼시는군요. 나와 자식들한테 빚을 떠넘기고 간 이 상황이 불만스러워. 지금 상황에서 벗어나기 위해 무엇을 할 수 있을까요? 상속 포기를 하면 빚을 갚지 않아도 된다는 얘기를 들은 거 같아. 변호사 도움을 받아야겠어. 변호사의 도움을 받아 문제가 해결되길 바라요.', '진짜 남편이 너무 한심해. 남편하게 한심함을 느끼시는군요. 남편이 담배를 못 끊어서 폐암 증세가 더 심해지고 있어. 남편의 흡연으로 병이 더 깊어졌군요. 어떻게 그럴 수가 있는지 내 생각으로는 이해할 수가 없어. 남편의 태도를 달라지게 할 방법이 없을까요?', '여자친구가 내가 좋아하는 것만큼 나를 사랑하지 않는 것 같아 속상해. 여자친구가 나를 사랑하지 않는 것 같아 슬프시군요. 나는 항상 먼저 배려하고 여자친구 입장에서 생각하는데 여자친구는 자신밖에 모르는 것 같아. 여자친구에게 보이는 배려가 보답받지 못하는 것 같아 서운하시겠어요. 여자친구한테 고민을 말해보았나요? 말해보았지만 반응이 시큰둥해. 크게 신경을 쓰는 것 같지 않아. 여자친구의 무덤덤한 반응이 나를 더 슬프게 만드는군요.']
[1, 1, 1, 1, 1]


### 토커나이저/모델 준비
- bert 한국어 버전 사전학습 모델 klue/bert-base

In [ ]:
from transformers import AutoModel, AutoTokenizer

model = AutoModel.from_pretrained('klue/bert-base')
tokenizer = AutoTokenizer.from_pretrained('klue/bert-base')

config.json:   0%|          | 0.00/425 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/289 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [ ]:
"""
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('klue/bert-base')
"""

"\nfrom transformers import BertTokenizer\n\ntokenizer = BertTokenizer.from_pretrained('klue/bert-base')\n"

In [ ]:
MAX_LEN = 30

X_train = tokenizer(X_train, padding=True, truncation=True, max_length=MAX_LEN)
X_test = tokenizer(X_test, padding=True, truncation=True, max_length=MAX_LEN)

X_train[:5], X_test[:5]

([Encoding(num_tokens=30, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
  Encoding(num_tokens=30, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
  Encoding(num_tokens=30, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
  Encoding(num_tokens=30, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
  Encoding(num_tokens=30, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])],
 [Encoding(num_tokens=30, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
  Encoding(num_tokens=30, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
  Encoding(num_tokens=30, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
  Encoding(num_tokens=3

In [ ]:
print(X_train['input_ids'][0])      # input_ids : 단어 토큰화
print(X_train['attention_mask'][0]) # attention_mask : 실제값/패딩 구분
print(X_train['token_type_ids'][0]) # token_type_ids : 첫번째/두번째 문장 구분 (단일 문장이라 다 0)

[2, 809, 2031, 721, 1160, 2460, 1295, 2169, 823, 2259, 568, 555, 2088, 721, 1465, 2205, 2259, 4307, 2052, 2275, 18, 3711, 1540, 27135, 3637, 3628, 2069, 1889, 2250, 3]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


### 데이터 파이프라인 생성

In [ ]:
type(X_train)

transformers.tokenization_utils_base.BatchEncoding

In [ ]:
import tensorflow as tf

train_ds = tf.data.Dataset.from_tensor_slices((dict(X_train), y_train))
test_ds = tf.data.Dataset.from_tensor_slices((dict(X_test), y_test))

In [ ]:
train_dataset = train_ds.shuffle(8000).batch(16).prefetch(tf.data.AUTOTUNE)
test_dataset = test_ds.batch(16).prefetch(tf.data.AUTOTUNE)

### 모델 로드 & 학습

In [ ]:
from transformers import TFBertForSequenceClassification

model = TFBertForSequenceClassification.from_pretrained('klue/bert-base', num_labels=2, from_pt=True)

pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import create_optimizer

num_train_steps = len(train_dataset) * 5
num_warmup_steps = int(num_train_steps * 0.1)

# optimizer
optimizer, _ = create_optimizer(
    init_lr=5e-5,                      # 학습률 초기값
    num_train_steps=num_train_steps,   # 총 학습 step수
    num_warmup_steps=num_warmup_steps, # warmup step수 (총 step수 1/10)
    weight_decay_rate=0.1              # 학습률 감쇠 계수
)

# model.hf_compute_loss == 데이터셋에 맞게 이진분류/다중분류 loss 함수 알아서 지정
model.compile(optimizer=optimizer, loss=model.hf_compute_loss, metrics=['accuracy'])

# 학습
model.fit(train_dataset, epochs=5, validation_data=test_dataset, batch_size=8)

Epoch 1/5
500/500 [==============================] - 120s 169ms/step - loss: 0.1662 - accuracy: 0.9451 - val_loss: 0.1118 - val_accuracy: 0.9547
Epoch 2/5
500/500 [==============================] - 83s 166ms/step - loss: 0.0730 - accuracy: 0.9784 - val_loss: 0.0894 - val_accuracy: 0.9713
Epoch 3/5
500/500 [==============================] - 84s 168ms/step - loss: 0.0381 - accuracy: 0.9893 - val_loss: 0.0781 - val_accuracy: 0.9747
Epoch 4/5
500/500 [==============================] - 84s 168ms/step - loss: 0.0142 - accuracy: 0.9964 - val_loss: 0.1005 - val_accuracy: 0.9743
Epoch 5/5
500/500 [==============================] - 84s 168ms/step - loss: 0.0039 - accuracy: 0.9991 - val_loss: 0.1090 - val_accuracy: 0.9757


### 모델 저장

In [ ]:
model.save_pretrained('AIhub1_model/bert-base')
tokenizer.save_pretrained('AIhub2_model/bert-base')

('AIhub2_model/bert-base/tokenizer_config.json',
 'AIhub2_model/bert-base/special_tokens_map.json',
 'AIhub2_model/bert-base/vocab.txt',
 'AIhub2_model/bert-base/added_tokens.json',
 'AIhub2_model/bert-base/tokenizer.json')

### 추론

In [ ]:
# 텍스트 분류 라벨 값 지정
model.config.id2label = {
    0: "긍정",
    1: "부정"
}

In [ ]:
from transformers import TextClassificationPipeline

sentiment_classifier = TextClassificationPipeline(
    tokenizer=tokenizer,
    model=model,
    framework='tf',
    # return_all_scores=True
)

Device set to use 0


In [ ]:
sentiment_classifier('직장 사람들과 좋은 관계를 가지시길 응원해요')

[{'label': '긍정', 'score': 0.987768828868866}]

In [ ]:
sentiment_classifier([
    '퇴사하고 싶어',
    '직장 사람들과 함께하는 것이 너무 즐거워',
    '오늘 날씨가 좋아서 기분이 좋아.',
    '비가와서 슬퍼',
    '야구장이 너무 시끄러워서 짜릿해'
])

[{'label': '부정', 'score': 0.999745786190033},
 {'label': '긍정', 'score': 0.9984161853790283},
 {'label': '긍정', 'score': 0.9989456534385681},
 {'label': '부정', 'score': 0.9995823502540588},
 {'label': '긍정', 'score': 0.8671702742576599}]

# HuggingFace

### push

In [ ]:
from google.colab import userdata

HF_TOKEN = userdata.get('HF_TOKEN')

In [ ]:
REPO_NAME = 'bert-base-practice'

model.push_to_hub(REPO_NAME, use_temp_dir=True, use_auth_token=HF_TOKEN)
tokenizer.push_to_hub(REPO_NAME, use_temp_dir=True, use_auth_token=HF_TOKEN)

### load

In [ ]:
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained('tokenizer/bert-base-practice')
model = AutoModel.from_pretrained('tf_model/bert-base-practice', from_tf=True)

### load한 모델 사용

In [ ]:
from transformers import pipeline

sentiment_classifier = pipeline('text-classification', model='Aihub/bert-base-practice')

In [ ]:
sentiment_classifier([
    '퇴근 후 여가에 회사 일을 더 열심히 해서 피해가 가지 않도록 해야겠어.',
    '내 능력이 부족한 거 같은데 그만 다녀야 될거같아.',
    '직장 동료와 상사들이랑 친하게 지내야겠어.',
    '직장 사람들과 좋은 관계를 가지시길 응원해요'
])